In [1]:
import os
os.environ["WANDB_MODE"]="disabled"
os.environ["CUDA_VISIBLE_DEVICE"]="0,1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
from tqdm import tqdm

import torch
from torch.nn import functional as F

import pandas as pd
from datasets import Dataset

import bitsandbytes as bnb
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, EarlyStoppingCallback
from trl import SFTTrainer
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

def get_prompt(data):
    prompts = []
    for (text, label) in zip(data["text"], data["sentiment_label"]):
        prompt = f"""<s>[INST] Generate the sentiment of the given text. Do not give explanation.\nText:{text}[/INST] {label} </s>"""
        prompts.append(prompt)
    return prompts

In [ ]:
train = pd.read_json("path_to_var_1_train_data", lines = True)
valid = pd.read_json("path_to_var_1_validation_data", lines = True)

train["prompt"] = get_prompt(train)
valid["prompt"] = get_prompt(valid)

train = Dataset.from_pandas(train[["prompt"]], split="train")
valid = Dataset.from_pandas(valid[["prompt"]], split="test")

In [ ]:
model_id = 'path_to_model_or_model_id'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
tokenizer.padding_side="left"
tokenizer.pad_token=tokenizer.eos_token

In [7]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'v_proj', 'up_proj']


In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=modules,
        task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [9]:
model.print_trainable_parameters()

trainable params: 95,420,416 || all params: 22,342,703,104 || trainable%: 0.4271


In [ ]:
args = TrainingArguments(
    output_dir="path_to_cache",
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=0.3,
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    overwrite_output_dir=True,
    lr_scheduler_type="constant",
    load_best_model_at_end = True
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="prompt",
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(3, 0.1)]
)

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,2.559700,2.373737
2,1.828000,2.502513
3,0.986300,2.882783
4,0.559500,3.320670


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=240, training_loss=1.4833824952443442, metrics={'train_runtime': 1575.3551, 'train_samples_per_second': 18.015, 'train_steps_per_second': 1.143, 'total_flos': 1.2511595900097331e+17, 'train_loss': 1.4833824952443442, 'epoch': 4.0})

In [ ]:
adaptor_dir = "path_to_output"

trainer.model.save_pretrained(adaptor_dir)

In [13]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

merged_model= PeftModel.from_pretrained(base_model, adaptor_dir)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
outs = []
test = pd.read_json("path_to_var_2_test_data", lines = True)


for text in tqdm(list(test["text"]), desc="Generating"):
    message = f"""<s>[INST] Generate the sentiment of the given text. Do not give explanation.\nText:{text}[/INST]"""
    encodeds = tokenizer(message, return_tensors="pt")
    model_inputs = encodeds.to(device)
    generated_ids = merged_model.generate(
        input_ids=encodeds["input_ids"],
        attention_mask=encodeds["attention_mask"],
        max_length=encodeds["input_ids"].shape[1] + 6,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.batch_decode(generated_ids)
    word = decoded[0].split("[/INST]", 1)[1].strip()
    outs.append(word)
    print(word)
    torch.cuda.empty_cache()
    del model_inputs
    del generated_ids
    del decoded
    del encodeds
    del word
    
pd.DataFrame({"prediction": outs}).to_json("path_to_results")

Generating:   0%|          | 1/469 [00:01<10:24,  1.33s/it]

1 1 1


Generating:   0%|          | 2/469 [00:02<09:51,  1.27s/it]

1 1 1


Generating:   1%|          | 3/469 [00:04<10:38,  1.37s/it]

...

1 1 1


Generating: 100%|█████████▉| 468/469 [09:42<00:01,  1.24s/it]

1 1 1


Generating: 100%|██████████| 469/469 [09:43<00:00,  1.24s/it]

0 0 0


In [ ]:
outs = []
test = pd.read_json("path_to_var_3_test_data", lines = True)


for text in tqdm(list(test["text"]), desc="Generating"):
    message = f"""<s>[INST] Generate the sentiment of the given text. Do not give explanation.\nText:{text}[/INST]"""
    encodeds = tokenizer(message, return_tensors="pt")
    model_inputs = encodeds.to(device)
    generated_ids = merged_model.generate(
        input_ids=encodeds["input_ids"],
        attention_mask=encodeds["attention_mask"],
        max_length=encodeds["input_ids"].shape[1] + 6,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.batch_decode(generated_ids)
    word = decoded[0].split("[/INST]", 1)[1].strip()
    outs.append(word)
    print(word)
    torch.cuda.empty_cache()
    del model_inputs
    del generated_ids
    del decoded
    del encodeds
    del word
    
pd.DataFrame({"prediction": outs}).to_json("path_to_results")

Generating:   0%|          | 1/517 [00:01<10:45,  1.25s/it]

1 1 1


Generating:   0%|          | 2/517 [00:02<10:33,  1.23s/it]

1 1 1


Generating:   1%|          | 3/517 [00:03<11:06,  1.30s/it]

...

1 1 1


Generating: 100%|█████████▉| 516/517 [10:53<00:01,  1.26s/it]

1 1 1


Generating: 100%|██████████| 517/517 [10:54<00:00,  1.27s/it]

1 1 1
